# Loading CSV files into SQLite (the CLI way)

#### Do we have any CSV files?

In [1]:
!ls -1 datasets/*.csv

datasets/Department.csv
datasets/Dependent.csv
datasets/Dept_Location.csv
datasets/Employee.csv
datasets/Project.csv
datasets/Works_On.csv


#### Great! How large are they?

In [2]:
!wc datasets/*.csv

       3       4     145 datasets/Department.csv
       7       8     306 datasets/Dependent.csv
       5       6      76 datasets/Dept_Location.csv
       9      18     704 datasets/Employee.csv
       6       7     184 datasets/Project.csv
      17      17     305 datasets/Works_On.csv
      47      60    1720 total


#### Let's look at the first couple of lines in each CSV file:

In [3]:
!head -n3 datasets/*.csv

==> datasets/Department.csv <==
Dname,Dnumber,Mgr_ssn,Mgr_start_date
Research,5,333445555,1988-05-22
Administration,4,987654321,1995-01-01

==> datasets/Dependent.csv <==
Essn,Dependent_name,Sex,Bdate,Relationship
333445555,Alice,F,1986-04-05,Daughter
333445555,Theodore,M,1983-10-25,Son

==> datasets/Dept_Location.csv <==
Dnumber,Dlocation
1,Houston
4,Stafford

==> datasets/Employee.csv <==
Fname,Minit,Lname,Ssn,Bdate,Address,Sex,Salary,Super_ssn,Dno
John,B,Smith,123456789,1965-01-09,"731 Fondren,Houston,TX",M,30000,333445555,5
Franklin,T,Wong,333445555,1955-12-08,"638 Voss,Houston,TX",M,40000,888665555,5

==> datasets/Project.csv <==
Pname,Pnumber,Plocation,Dnum
ProductX,1,Bellaire,5
ProductY,2,Sugarland,5

==> datasets/Works_On.csv <==
Essn,Pno,Hours
123456789,1,32.5
123456789,2,7.5


#### Let's review some SQLite-specific "dot" commands for importing CSV files directly

In [4]:
%%file load-csv-files.sql

-- We want to use CSV (comma-separated values) files: 
.mode csv 

-- Import csv-file into a table
.import datasets/Employee.csv Employee
.import datasets/Department.csv Department
.import datasets/Dependent.csv Dependent
.import datasets/Dept_Location.csv Dept_Location
.import datasets/Project.csv Project
.import datasets/Works_On.csv Works_On

-- make output prettier 
.mode columns
.headers on
.echo on

-- Display the database schema (table names only)
.tables 

-- Show the number of rows for each table:

SELECT count(*) FROM Employee;

SELECT count(*) FROM Department;

SELECT count(*) FROM Dependent;

SELECT count(*) FROM Dept_Location;

SELECT count(*) FROM Project;

-- SELECT count(*) FROM Works_on;

-- Show the first couple of rows of each table:

SELECT * FROM Employee LIMIT 3; 

SELECT * FROM Department LIMIT 3;

SELECT * FROM Dependent LIMIT 3;

SELECT * FROM Dept_Location LIMIT 3;

SELECT * FROM Project LIMIT 3;

SELECT * FROM Works_on LIMIT 3;

.schema

.exit


Overwriting load-csv-files.sql


#### Remove the COMPANY database file (if it exists)

In [5]:
!rm COMPANY.db

#### Create COMPANY database by running the load script

In [6]:
!sqlite3 COMPANY.db < load-csv-files.sql


-- Display the database schema (table names only)
.tables 
Department     Dept_Location  Project      
Dependent      Employee       Works_On     

-- Show the number of rows for each table:

SELECT count(*) FROM Employee;
count(*)
--------
8       

SELECT count(*) FROM Department;
count(*)
--------
3       

SELECT count(*) FROM Dependent;
count(*)
--------
7       

SELECT count(*) FROM Dept_Location;
count(*)
--------
5       

SELECT count(*) FROM Project;
count(*)
--------
6       

-- SELECT count(*) FROM Works_on;

-- Show the first couple of rows of each table:

SELECT * FROM Employee LIMIT 3; 
Fname     Minit  Lname   Ssn        Bdate       Address                 Sex  Salary  Super_ssn  Dno
--------  -----  ------  ---------  ----------  ----------------------  ---  ------  ---------  ---
John      B      Smith   123456789  1965-01-09  731 Fondren,Houston,TX  M    30000   333445555  5  
Franklin  T      Wong    333445555  1955-12-08  638 Voss,Houston,TX     M    40000   888

#### Uhh.. that schema looks a bit ugly. We need to edit this .. 

#### Let's create a SQL "dump", then edit it as needed

In [7]:
!sqlite3 COMPANY.db .dump > COMPANY-dump.sql

In [8]:
!head COMPANY-dump.sql

PRAGMA foreign_keys=OFF;
BEGIN TRANSACTION;
CREATE TABLE IF NOT EXISTS "Employee"(
"Fname" TEXT, "Minit" TEXT, "Lname" TEXT, "Ssn" TEXT,
 "Bdate" TEXT, "Address" TEXT, "Sex" TEXT, "Salary" TEXT,
 "Super_ssn" TEXT, "Dno" TEXT);
INSERT INTO Employee VALUES('John','B','Smith','123456789','1965-01-09','731 Fondren,Houston,TX','M','30000','333445555','5');
INSERT INTO Employee VALUES('Franklin','T','Wong','333445555','1955-12-08','638 Voss,Houston,TX','M','40000','888665555','5');
INSERT INTO Employee VALUES('Alicia','J','Zelaya','999887777','1968-01-19','3321 Castle,Spring,TX','F','25000','987654321','4');
INSERT INTO Employee VALUES('Jennifer','S','Wallace','987654321','1941-06-20','291 Berry,Bellaire,TX','F','43000','888665555','4');


#### After editing it might look like this:

In [9]:
%%file COMPANY-schema-edited.sql

-- NOTE: With Foreign Keys set to ON, we have a problem. Why??
PRAGMA foreign_keys=OFF;

CREATE TABLE Employee(
       Fname TEXT,
       Minit TEXT,
       Lname TEXT,
       Ssn PRIMARY KEY,
       Bdate TEXT,
       Address TEXT,
       Sex TEXT,
       Salary TEXT,
       Super_ssn TEXT REFERENCES Employee ON DELETE CASCADE, -- careful here! Do we really want this? 
       Dno INTEGER REFERENCES Department
       );

INSERT INTO Employee VALUES('James','E','Borg','888665555','1937-11-10','450 Stone,Houston,TX','M','55000',NULL,'1');
INSERT INTO Employee VALUES('John','B','Smith','123456789','1965-01-09','731 Fondren,Houston,TX','M','30000','333445556','5');
INSERT INTO Employee VALUES('Franklin','T','Wong','333445555','1955-12-08','638 Voss,Houston,TX','M','40000','888665555','5');
INSERT INTO Employee VALUES('Alicia','J','Zelaya','999887777','1968-01-19','3321 Castle,Spring,TX','F','25000','987654321','4');
INSERT INTO Employee VALUES('Jennifer','S','Wallace','987654321','1941-06-20','291 Berry,Bellaire,TX','F','43000','888665555','4');
INSERT INTO Employee VALUES('Ramesh','K','Narayan','666884444','1962-09-15','975 Fire Oak,Humble,TX','M','38000','333445555','5');
INSERT INTO Employee VALUES('Joyce','A','English','453453453','1972-07-31','5631 Rice,Houston,TX','F','25000','333445555','5');
INSERT INTO Employee VALUES('Ahmad','V','Jabbar','987987987','1969-03-29','980 Dallas,Houston,TX','M','25000','987654321','4');


CREATE TABLE Department(
       Dname TEXT,
       Dnumber INTEGER PRIMARY KEY,
       Mgr_ssn TEXT REFERENCES Employee,
       Mgr_start_date TEXT
       );

INSERT INTO Department VALUES('Research','5','333445555','1988-05-22');
INSERT INTO Department VALUES('Administration','4','987654321','1995-01-01');
INSERT INTO Department VALUES('Headquarters','1','888665555','1981-06-19');


CREATE TABLE Dependent(
       Essn TEXT REFERENCES Employee,
       Dependent_name TEXT,
       Sex TEXT,
       Bdate TEXT,
       Relationship TEXT
       );

INSERT INTO Dependent VALUES('333445555','Alice','F','1986-04-05','Daughter');
INSERT INTO Dependent VALUES('333445555','Theodore','M','1983-10-25','Son');
INSERT INTO Dependent VALUES('333445555','Joy','F','1958-05-03','Spouse');
INSERT INTO Dependent VALUES('987654321','Abner','M','1942-02-28','Spouse');
INSERT INTO Dependent VALUES('123456789','Michael','M','1988-01-04','Son');
INSERT INTO Dependent VALUES('123456789','Alice','F','1988-12-30','Daughter');
INSERT INTO Dependent VALUES('123456789','Elizabeth','F','1967-05-05','Spouse');

CREATE TABLE Dept_Location(
       Dnumber INTEGER REFERENCES Department,
       Dlocation TEXT
       );

INSERT INTO Dept_Location VALUES('1','Houston');
INSERT INTO Dept_Location VALUES('4','Stafford');
INSERT INTO Dept_Location VALUES('5','Bellaire');
INSERT INTO Dept_Location VALUES('5','Sugarland');
INSERT INTO Dept_Location VALUES('5','Houston');

CREATE TABLE Project(
       Pname TEXT,
       Pnumber INTEGER PRIMARY KEY,
       Plocation TEXT,
       Dnum INTEGER REFERENCES Department
       );

INSERT INTO Project VALUES('ProductX','1','Bellaire','5');
INSERT INTO Project VALUES('ProductY','2','Sugarland','5');
INSERT INTO Project VALUES('ProductZ','3','Houston','5');
INSERT INTO Project VALUES('Computerization','10','Stafford','4');
INSERT INTO Project VALUES('Reorganization','20','Houston','1');
INSERT INTO Project VALUES('Newbenefits','30','Stafford','4');

CREATE TABLE Works_On(
       Essn TEXT REFERENCES Employee,
       Pno INTEGER REFERENCES Project,
       Hours TEXT
       );

INSERT INTO Works_On VALUES('123456789','1','32.5');
INSERT INTO Works_On VALUES('123456789','2','7.5');
INSERT INTO Works_On VALUES('666884444','3','40.0');
INSERT INTO Works_On VALUES('453453453','1','20.0');
INSERT INTO Works_On VALUES('453453453','2','20.0');
INSERT INTO Works_On VALUES('333445555','2','10.0');
INSERT INTO Works_On VALUES('333445555','3','10.0');
INSERT INTO Works_On VALUES('333445555','10','10.0');
INSERT INTO Works_On VALUES('333445555','20','10.0');
INSERT INTO Works_On VALUES('999887777','30','30.0');
INSERT INTO Works_On VALUES('999887777','10','10.0');
INSERT INTO Works_On VALUES('987987987','10','35.0');
INSERT INTO Works_On VALUES('987987987','30','5.0');
INSERT INTO Works_On VALUES('987654321','30','20.0');
INSERT INTO Works_On VALUES('987654321','20','15.0');
INSERT INTO Works_On VALUES('888665555','20',NULL);

PRAGMA foreign_keys=ON;

.schema


Overwriting COMPANY-schema-edited.sql


#### Let's rebuild the database from this SQL file

In [10]:
!rm COMPANY.db
!sqlite3 COMPANY.db < COMPANY-schema-edited.sql

CREATE TABLE Employee(
       Fname TEXT,
       Minit TEXT,
       Lname TEXT,
       Ssn PRIMARY KEY,
       Bdate TEXT,
       Address TEXT,
       Sex TEXT,
       Salary TEXT,
       Super_ssn TEXT REFERENCES Employee ON DELETE CASCADE, -- careful here! Do we really want this? 
       Dno INTEGER REFERENCES Department
       );
CREATE TABLE Department(
       Dname TEXT,
       Dnumber INTEGER PRIMARY KEY,
       Mgr_ssn TEXT REFERENCES Employee,
       Mgr_start_date TEXT
       );
CREATE TABLE Dependent(
       Essn TEXT REFERENCES Employee,
       Dependent_name TEXT,
       Sex TEXT,
       Bdate TEXT,
       Relationship TEXT
       );
CREATE TABLE Dept_Location(
       Dnumber INTEGER REFERENCES Department,
       Dlocation TEXT
       );
CREATE TABLE Project(
       Pname TEXT,
       Pnumber INTEGER PRIMARY KEY,
       Plocation TEXT,
       Dnum INTEGER REFERENCES Department
       );
CREATE TABLE Works_On(
       Essn TEXT REFERENCES Employee,
       Pno INTEGER REFERENC

#### Looking good. We could do more fancy things with the schema.. But on to queries instead!

In [11]:
%%file simple-queries.sql

-- Do nice columnar output
.mode columns

.print 'Q1: How many employees do we have?'
.print ''

SELECT COUNT(*) as '#Employees' FROM Employee;

.print ''
.print 'Q2: Who is the boss? The manager of the department that an employee works for!'
.print ''

SELECT E.SSN as 'Employee', 
       M.SSN as 'Boss'
FROM   Employee E, 
       Employee M
WHERE E.Super_SSN = M.SSN
ORDER BY M.SSN;

.print ''
.print 'Q3: OK, we need names:'
.print ''

SELECT E.LName || ', ' || E.FName as 'Employee', 
       M.LName || ', ' || M.FName as 'Boss'
FROM   Employee E, 
       Employee M
WHERE E.Super_SSN = M.SSN;

Overwriting simple-queries.sql


#### Now running those simple queries:

In [12]:
!sqlite3 COMPANY.db < simple-queries.sql

Q1: How many employees do we have?

#Employees
----------
8         

Q2: Who is the boss? The manager of the department that an employee works for!

Employee   Boss     
---------  ---------
666884444  333445555
453453453  333445555
333445555  888665555
987654321  888665555
999887777  987654321
987987987  987654321

Q3: OK, we need names:

Employee           Boss             
-----------------  -----------------
Wong, Franklin     Borg, James      
Zelaya, Alicia     Wallace, Jennifer
Wallace, Jennifer  Borg, James      
Narayan, Ramesh    Wong, Franklin   
English, Joyce     Wong, Franklin   
Jabbar, Ahmad      Wallace, Jennifer


# SQL Exercises

### Query Q4: Retrieve the name and address of all employees who work for the "Administration" department.

In [13]:
%%file week9-Q4.sql

.mode columns

.print 'Q4: Retrieve the name and address of all employees who work for the "Administration" department.'
.print ''

SELECT FName, Minit, Lname, Address 
FROM   Employee E, Department D
WHERE  E.Dno = D.Dnumber AND D.Dname = 'Administration';

Overwriting week9-Q4.sql


In [14]:
!sqlite3 COMPANY.db < week9-Q4.sql

Q4: Retrieve the name and address of all employees who work for the "Administration" department.

Fname     Minit  Lname    Address              
--------  -----  -------  ---------------------
Alicia    J      Zelaya   3321 Castle,Spring,TX
Jennifer  S      Wallace  291 Berry,Bellaire,TX
Ahmad     V      Jabbar   980 Dallas,Houston,TX


### Substring Pattern Matching and Arithmetic Operators
- Using the **LIKE** comparison operator for pattern matching
- Partial strings are specified using two reserved characters:
  > % replaces an arbitrary number of zero or more characters
  
  > underscore (_) replaces a single character

### Query Q5: Retrieve all project names that include the string "Product".

In [15]:
%%file week9-Q5.sql

.mode columns

.print 'Q5: Retrieve all project names that include the string "Product".'
.print ''

# Insert your SQL query here:
SELECT Pname
FROM   Project
WHERE  Pname LIKE '%Product%';

Overwriting week9-Q5.sql


In [16]:
!sqlite3 COMPANY.db < week9-Q5.sql

Q5: Retrieve all project names that include the string "Product".

Pname   
--------
ProductX
ProductY
ProductZ


## Nested Queries 

### Query Q6: Retrieve the name of each employee who has a dependent with the same sex as the employee.

In [17]:
%%file week9-Q6.sql

.mode columns

.print 'Q6: Retrieve the name of each employee who has a dependent with the same sex as the employee.'
.print ''

# Insert your SQL query here:
SELECT E.Fname, E.Lname
FROM   Employee AS E
WHERE  E.Ssn IN (SELECT D.Essn
                FROM Dependent AS D
                WHERE E.Sex=D.Sex);

.print ''
.print '... same as before, but with a simple join and report also dependent names'
.print ''

# Insert your SQL query here:
SELECT E.Fname, E.Lname, D.Dependent_name 
FROM   Employee E, Dependent D
WHERE E.Ssn = D.Essn AND E.Sex=D.Sex;
    

Overwriting week9-Q6.sql


In [18]:
!sqlite3 COMPANY.db < week9-Q6.sql

Q6: Retrieve the name of each employee who has a dependent with the same sex as the employee.

Fname     Lname
--------  -----
John      Smith
Franklin  Wong 

... same as before, but with a simple join and report also dependent names

Fname     Lname  Dependent_name
--------  -----  --------------
Franklin  Wong   Theodore      
John      Smith  Michael       


## Aggregate Functions in SQL 
- Aggregate functions are used to summarize information from multiple tuples into a single-tuple summary.
  > Grouping is used to create subgroups of tuples before summarization.
  
  > A number of built-in aggregate functions exist: COUNT, SUM, MAX, MIN, and AVG.
  
  > HAVING clause 

### Query Q7: Find the sum of the salaries, the maximum salary, and the minimum salary for all female employees.

In [19]:
%%file week9-Q7.sql

.mode columns

.print 'Q7: Find the sum of the salaries, the maximum salary, and the minimum salary for all female employees.'
.print ''

# Insert your SQL query here:
SELECT SUM(Salary) as 'Total_salary', MAX(Salary) as 'Max_salary', MIN(Salary) as 'Min_salary'
FROM   Employee
WHERE  sex = 'F';


Overwriting week9-Q7.sql


In [20]:
!sqlite3 COMPANY.db < week9-Q7.sql

Q7: Find the sum of the salaries, the maximum salary, and the minimum salary for all female employees.

Total_salary  Max_salary  Min_salary
------------  ----------  ----------
93000         43000       25000     


### Query Q8: Retrieve the total number of employees in the "Research" department. 

In [21]:
%%file week9-Q8.sql

.mode columns

.print 'Q8: Retrieve the total number of employees in the "Research" department. '
.print ''

# Insert your SQL query here:
SELECT  COUNT(*) as 'Number of Employees'
FROM Employee E, Department D
WHERE E.Dno = D.Dnumber AND
      D.Dname = 'Research';
    

Overwriting week9-Q8.sql


In [22]:
!sqlite3 COMPANY.db < week9-Q8.sql

Q8: Retrieve the total number of employees in the "Research" department. 

Number of Employees
-------------------
4                  


In [5]:
%%file week9-Q9.sql

.mode columns

.print 'Q9: For each depar....'
.print ''

# Insert your SQL query here:
SELECT  Dno, COUNT(*) as 'Number of Employees'
FROM Employee E, Department D
GROUP BY Dno;
    

Overwriting week9-Q9.sql


In [6]:
!sqlite3 COMPANY.db < week9-Q9.sql

Q9: ....

Dno  Number of Employees
---  -------------------
1    3                  
4    9                  
5    12                 
